## Execute GNN model

Test imported GNN models

In [11]:
import torch
from torch_geometric.utils import k_hop_subgraph
from torch_geometric.datasets import BAShapes


## load a BAshapes dataset
dataset = BAShapes(connection_distribution="random")
print(f"[dataset]> ...loading dataset '{dataset}' from PyG")

graph = dataset[0]
print(f"\n[dataset]> {dataset} dataset graph...")
print("\t->", graph)


## extract a random node to train on
l = range(400, 700, 5)
idx = torch.randint(0, len(l), (1,))
node_idx = torch.tensor([l[idx]]) 
print(f"\nChosing node {node_idx.item()}...")

edge_idx = graph.edge_index
_, sub_index, _, _ = k_hop_subgraph(node_idx, 3, edge_idx)
print("\tnode neighborhood:", sub_index.size())

## node features
x = graph.x

[dataset]> ...loading dataset 'BAShapes()' from PyG

[dataset]> BAShapes() dataset graph...
	-> Data(x=[700, 10], edge_index=[2, 3938], y=[700], expl_mask=[700], edge_label=[3938])

Chosing node 515...
	node neighborhood: torch.Size([2, 550])


In [12]:
import os
root_path = os.getcwd()
print(root_path)

from gnns.GNNpaper import GraphGCN, NodeGCN

#### instantiate GNNs model
model = NodeGCN(num_features=10, num_classes=4, device="cpu")
output = model(x, sub_index)[node_idx]
print(f"\nNodeGCN output for node {node_idx.item()}:\n\t", output)


/media/zascerta/DATA/Universalis/eng_computer_science/thesis/code/bardocchiolo/src

NodeGCN output for node 515:
	 tensor([[-0.0743, -0.1241,  0.1271, -0.1530]], grad_fn=<IndexBackward0>)


In [13]:
from gnns.CFGNNpaper.gcn import GCNSynthetic

#### need dense adjacency matrix for GCNSynthetic model
v = torch.ones(sub_index.size(1))
s = (graph.num_nodes,graph.num_nodes)
sub_index = torch.sparse_coo_tensor(indices=sub_index, values=v, size=s).to_dense()

model = GCNSynthetic(nfeat=10,nhid=20,nout=20,nclass=4,dropout=0.0)
output = model(x, sub_index)[node_idx]
print(f"GCNSynthetic output for node {node_idx.item()}:\n\t", output)

GCNSynthetic output for node 515:
	 tensor([[-0.6209, -1.0587, -2.7275, -2.9901]], grad_fn=<IndexBackward0>)
